In [6]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [7]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [8]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [9]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 967 images belonging to 12 classes.
Found 221 images belonging to 12 classes.


In [10]:
# Define the VGG16 base model
def create_EfficientNetV2L_model():
    EfficientNetV2L_model = EfficientNetV2L(
        weights='imagenet', 
        include_top=False, 
        input_shape=(img_height, img_width, 3)
    )
    EfficientNetV2L_model.trainable = False


    model = Sequential([
        EfficientNetV2L_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(12, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [11]:
#MODEL fit
model = create_EfficientNetV2L_model()
model.fit(train_generator, epochs=10, validation_data=test_generator)


473176280/473176280 [==============================] - 83s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Functiona  (None, 4, 4, 1280)       117746848 
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                81984     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)               

In [12]:
#predict
predictions = model.predict(test_generator)



7/7 [==============================] - 19s 290ms/step


In [13]:
#saving predictions in csv file
np.savetxt("../model_saved_files/predictions/EfficientNetV2L_predictions.csv", predictions, delimiter=",")

# Save the model
# model.save("../model_saved_files/models_weights/EfficientNetV2L.h5")